In [1]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import pickle

ImportError: DLL load failed: Det går inte att hitta den angivna proceduren.

In [25]:
SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [26]:
def load_data():
    
    return pickle.load(open('df_down_sampled_alternative.p', 'rb'))

In [27]:
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

In [28]:
df = load_data()

In [29]:
df.fillna(df.mean(), inplace = True)

In [30]:
#Action is the target-variable. Equals 1 if resp is positive, 0 otherwise!

df['action'] = ((df['resp'].values) > 0).astype(int)

In [31]:
features = [c for c in df.columns if "feature" in c]

In [32]:
f_mean = np.mean(df[features[1:]].values, axis=0)

In [33]:
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [34]:
X_train = df.loc[:, df.columns.str.contains('feature')]

In [35]:
X_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
527894,1,3.151305,5.467693,-0.164505,-0.189219,0.663966,0.988896,0.038627,0.013857,2.184804,...,0.367166,0.188722,2.433699,4.282284,1.621115,4.331030,2.553220,3.799011,2.642943,3.998054
527896,-1,-0.365888,0.824004,-0.293208,-0.416391,-0.599185,-0.997330,0.038627,0.013857,-0.869330,...,0.367166,0.188722,-0.702873,4.038753,-0.789767,4.133183,-1.207878,3.402796,-0.928290,3.511141
527897,1,1.514607,0.596214,0.324062,0.154730,0.845069,0.521491,0.038627,0.013857,0.310387,...,0.367166,0.188722,2.304354,1.530169,3.596848,4.613493,4.516110,3.341374,2.635798,1.535235
527900,-1,-1.008999,0.242132,-0.076187,-0.073584,-0.330932,-0.634109,0.038627,0.013857,-2.315807,...,0.367166,0.188722,-1.700655,-0.376747,-0.876730,1.645537,-1.318172,1.369452,-1.352577,0.733124
527901,-1,-0.391784,0.152261,-0.603803,-0.711527,-1.204166,-1.579647,0.038627,0.013857,-1.926413,...,0.367166,0.188722,-1.690686,1.656818,-1.213169,2.949877,-2.033542,2.248799,-1.838696,1.905638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390444,-1,1.538675,2.530447,2.494852,3.263345,1.613620,2.097220,-0.401539,-0.489412,-0.045341,...,-1.872247,-2.084489,-0.984942,1.129901,-1.632432,-2.169964,-2.371293,-0.889212,-1.554352,0.215984
2390446,1,0.270380,-1.231874,-5.802676,-3.172423,-4.357278,-2.301009,1.957683,1.000846,4.245754,...,1.210442,-1.982950,1.724863,-0.984278,1.413825,-1.598825,2.087731,-1.126050,1.590538,-1.250209
2390478,-1,-0.134380,0.160580,1.292513,1.453954,0.605912,0.687598,-0.489143,-0.593642,-0.915392,...,-0.342937,-2.103206,-0.765664,-2.148415,-0.599358,-3.155134,-0.957971,-2.285314,-0.894580,-2.064227
2390481,-1,-0.779554,-0.597258,0.674234,0.735692,-0.153732,-0.165179,-0.175335,-0.193784,-0.801560,...,0.053802,-3.453253,1.173186,-1.215499,0.170404,-3.433334,0.496345,-2.224752,1.207851,-1.264984


In [36]:
batch_size = 5000
hidden_units = [150, 150, 150]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

In [37]:
clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

AttributeError: 'google.protobuf.pyext._message.RepeatedCompositeCo' object has no attribute 'append'

In [23]:
tf.__version__

'2.4.0'